#Продвинутый Python, ДЗ-2

Правила игры:

В домашке 8 задач, разбаловка указана в задании. Суммарно за дз можно получить 100 баллов, что равняется 10 баллам

В каждой задаче необходимо реализовать функцию, которая после будет проверяться через github classroom на тестах. Сами тесты лежит в гитхабе, можете локально проверить работу функций перед сдачей

Дедлайн - 7 дней после выдачи дз. Необходимо залить решеннный ноутбук в github и прислать ссылку в Anytask (без выполнения любого из пунктов работа проверяться не будет)

В данной домашке нужно использовать pandas

In [10]:
# https://stackoverflow.com/a/60658965/7286121

from IPython.core.magic import register_cell_magic

@register_cell_magic
def write_and_run(line, cell):
    argz = line.split()
    file = argz[-1]
    mode = 'w'
    if len(argz) == 2 and argz[0] == '-a':
        mode = 'a'
    with open(file, mode) as f:
        f.write(cell)
    get_ipython().run_cell(cell)

## Данные

В этом домашнем задании вам придется оказаться на месте аналитика в бразильском маркетплейсе [Olist](https://olist.com/pt-br/). Вам необходимо исследовать данные и на их основании сделать выводы, которые помогут бизнесу расцветать!

Данные находятся тут в файле archive.zip

In [11]:
#!pip install wget
#перед сдачей это закомментить

  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9675 sha256=712606e1e7c3d73e5f7b522d70fd88c5e63924c23151e8bb3c70e6d21aab31d4
  Stored in directory: /Users/marinasidorenko/Library/Caches/pip/wheels/04/5f/3e/46cc37c5d698415694d83f607f833f83f0149e49b3af9d0f38
Successfully built wget


In [12]:
# Код, который будет в каждом тесте, названия не менять
import pandas as pd
import numpy as np
import zipfile
import wget

url = 'https://github.com/Palladain/Deep_Python/raw/main/Homeworks/Homework_1/archive.zip'
filename = wget.download(url)

with zipfile.ZipFile(filename, 'r') as zip_ref:
    zip_ref.extractall('./')

customers = pd.read_csv('olist_customers_dataset.csv')
location = pd.read_csv('olist_geolocation_dataset.csv')
items = pd.read_csv('olist_order_items_dataset.csv')
payments = pd.read_csv('olist_order_payments_dataset.csv')
reviews = pd.read_csv('olist_order_reviews_dataset.csv')
orders = pd.read_csv('olist_orders_dataset.csv')
products = pd.read_csv('olist_products_dataset.csv')
translation = pd.read_csv('product_category_name_translation.csv')
sellers = pd.read_csv('olist_sellers_dataset.csv')

  1% [.                                                   ]   860160 / 44717580

  4% [..                                                  ]  1949696 / 44717580

 11% [.....                                               ]  5070848 / 44717580

 16% [........                                            ]  7471104 / 44717580

 22% [...........                                         ]  9887744 / 44717580

...

 87% [.............................................       ] 39272448 / 44717580

 90% [..............................................      ] 40329216 / 44717580

 95% [.................................................   ] 42541056 / 44717580

 97% [..................................................  ] 43769856 / 44717580

100% [....................................................] 44717580 / 44717580

Ван дано 9 датасетов, которые содержат в себе все данные по 100 000 заказам со всей Бразилии. Чтобы облегчить вам жизнь, вот связи по этим датасетам (файл product_category_name_translation является переводом названий категорий с португальского на английский)

![](https://i.imgur.com/HRhd2Y0.png)

Ну что же, начнем!

## Задание 1 (10 баллов)

Определите:

* Число товаров

* Среднюю стоимость товара (стоимость товара = среднее от цен в датасете items)

в разрезе категорий (все категории должны быть на английском языке)

Табличка, которая у вас должна получиться:

```
category | products | price

value    | value      | value
```

**Обратите внимание:**

Для категории portateis_cozinha_e_preparadores_de_alimentos перевод portable kitchen and food preparers

Для категории pc_gamer перевод PC Gamer

Для них нужно отдельно добавить перевод

In [13]:
%%write_and_run task_1.py

# важно! все зависимости, которые используете (если добавляее новые) в этом классе надо явно продублировать в эту ячейку
import numpy as np
import pandas as pd

def task_1(translation, items, products):
    items_with_categories = items.join(products.set_index('product_id')[['product_category_name']], on='product_id')
    items_with_categories = items_with_categories.join(translation.set_index('product_category_name')[['product_category_name_english']], on='product_category_name')
    items_with_categories.loc[items_with_categories['product_category_name'] == 'portateis_cozinha_e_preparadores_de_alimentos', 'product_category_name_english'] = 'portable kitchen and food preparers'
    items_with_categories.loc[items_with_categories['product_category_name'] == 'pc_gamer', 'product_category_name_english'] = 'PC Gamer'
    df = items_with_categories.groupby(["product_id", "product_category_name_english"]).agg({"price" : "mean"}).reset_index()
    df = df.groupby(["product_category_name_english"]).agg({"product_id": "count", "price" : "mean"}).reset_index()
    df = df.rename(columns={"product_category_name_english": "category", "product_id": "products"})
    return df

In [14]:
#Проверки

res = task_1(translation, items, products)

assert res[res.category == 'portable kitchen and food preparers'].price.values[0] == 186.996
assert len(res) == 73
assert len(res.drop_duplicates()) == 73
assert res[res.category == 'drinks'].products.values[0] == 81
assert res.products.sum() == 32341
assert res.price.sum() == 12459.751444351941
assert res[res.category == 'home_confort'].price.values[0] == 185.56926417326417

## Задание 2 (10 баллов)

Определите для каждого продавца основную категорию их продаж (категории должны быть на английском языке)

Табличка, которая у вас должна получиться:

```
seller_id | category

value    | value
```




In [15]:
%%write_and_run task_2.py

# важно! все зависимости, которые используете (если добавляее новые) в этом классе надо явно продублировать в эту ячейку
import numpy as np
import pandas as pd

def task_2(translation, products, items):
    items_with_categories = items.join(products.set_index('product_id')[['product_category_name']], on='product_id')
    items_with_categories = items_with_categories.join(translation.set_index('product_category_name')[['product_category_name_english']], on='product_category_name')
    items_with_categories.loc[items_with_categories['product_category_name'] == 'portateis_cozinha_e_preparadores_de_alimentos', 'product_category_name_english'] = 'portable kitchen and food preparers'
    items_with_categories.loc[items_with_categories['product_category_name'] == 'pc_gamer', 'product_category_name_english'] = 'PC Gamer'
    df = items_with_categories.groupby(['seller_id', 'product_category_name_english']).agg({'order_id' : 'count'}).reset_index()
    df = df.sort_values('order_id', ascending = False).drop_duplicates('seller_id')
    df = df.drop(columns = 'order_id')
    df = df.rename(columns={"product_category_name_english": "category"})
    return df

In [16]:
# Проверки

res = task_2(translation, products, items)
assert res[res.seller_id == 'e3e15e2c0b9700561efac21c6be48066'].category.values[0] == 'housewares'
assert res[res.seller_id == '2f73e04d12cdf0c945ded66bb3fcf6c7'].category.values[0] == 'garden_tools'
assert len(res) == len(res.drop_duplicates())
assert len(res) == 3035
assert len(res[res.category == 'telephony']) == 66
assert list(np.sort(res.groupby("category").agg({"seller_id": "nunique"}).seller_id.values)) == [  1,   1,   1,   1,   1,   2,   2,   2,   2,   2,   3,   3,   4,
         4,   4,   4,   5,   5,   5,   5,   5,   5,   6,   6,   6,   7,
         8,  10,  12,  13,  13,  13,  14,  14,  14,  15,  16,  17,  17,
        17,  19,  20,  20,  20,  21,  22,  26,  37,  37,  43,  46,  51,
        54,  59,  66,  66,  78,  87,  87,  99, 101, 116, 125, 156, 216,
       224, 256, 288, 310]

## Задание 3 (10 баллов)

Выведите долю потраченных денег по каждому штату (потраченные деньги - сумма денег по доставленным заказам, сумма по price и freight_value)

*Примечание:* разбивка по штатам - по штату покупателя, процент - число от 0 до 1


Табличка, которая у вас должна получиться:

```
state | perc

value | value
```

In [17]:
%%write_and_run task_3.py

# важно! все зависимости, которые используете (если добавляее новые) в этом классе надо явно продублировать в эту ячейку
import numpy as np
import pandas as pd

def task_3(orders, customers, items):
    orders_full = orders.join(items.set_index('order_id')[['freight_value', 'price']], on='order_id')
    orders_full = orders_full.join(customers.set_index('customer_id')[['customer_state']], on = 'customer_id')
    orders_full = orders_full[orders_full['order_status'] == 'delivered']
    orders_full['cost'] = orders_full['price'] + orders_full['freight_value']
    df = orders_full.groupby(['customer_state']).agg({'cost': 'sum'}).reset_index()
    total_sum = df['cost'].sum()
    df['perc'] = (df['cost']).apply(lambda x : x / total_sum)
    df = df.drop(columns = ['cost'])
    df = df.rename(columns={"customer_state": "state"})
    return df

In [18]:
# Проверки

res = task_3(orders, customers, items)
assert res.perc.sum() == 1
assert res[res.state == "RS"].perc.values[0] == 0.055868056429816286
assert res.sort_values("perc", ascending=True).iloc[0, 1] == 0.0005862290943146945
assert res.sort_values("perc", ascending=False).iloc[0, 1] == 0.3741756035817322
assert len(res) == 27

## Задание 4 (10 баллов)

Определите средний чек покупки (добавьте разбивку на стоимость самого заказ и стоимость доставки) и среднее число товаров в заказе

А также определите среднее число покупок на пользователя (обратите внимание на идентификаторы)

In [28]:
%%write_and_run task_4.py

# важно! все зависимости, которые используете (если добавляее новые) в этом классе надо явно продублировать в эту ячейку
import numpy as np
import pandas as pd

def task_4(items, orders, customers):
    orders_full = orders.join(items.set_index('order_id')[['freight_value', 'price', 'product_id', 'order_item_id']], on='order_id')
    orders_full = orders_full.join(customers.set_index('customer_id')[['customer_state', 'customer_unique_id']], on = 'customer_id')
    df = items.groupby(['order_id']).agg({'price': 'sum', 'freight_value' : 'sum'}).reset_index()
    receipt_price = df['price'].mean()
    receipt_delivery = df['freight_value'].mean()
    df = items.groupby(['order_id']).agg({'product_id' : 'count'}).reset_index()
    quantity = df['product_id'].mean()
    df = orders_full.groupby(['customer_unique_id']).agg({'order_id' : 'nunique'}).reset_index()
    purchased = df['order_id'].mean()
    return (receipt_price, receipt_delivery, quantity, purchased)

In [30]:
# Проверка

res = task_4(items, orders, customers)
assert res == (137.7540763788945, 22.823561713254815, 1.1417306873695092, 1.0348089410589412)

## Задание 5 (10 баллов)

Посчитайте CSAT (customer satisfaction - средняя оценка ревью) и отобразите средний CSAT по дням в период с апреля 2017 по апрель 2018 года

Все манипуляции с датой необходимо сделать с помощью datetime и dateutil


Табличка, которая у вас должна получиться:

```
date | csat

"YYYY-MM-DD" | value
```

In [26]:
%%write_and_run task_5.py

# важно! все зависимости, которые используете (если добавляее новые) в этом классе надо явно продублировать в эту ячейку
import numpy as np
import pandas as pd
from dateutil.parser import parse
import datetime

def task_5(reviews):
    df = reviews.copy()
    df = reviews.groupby(['review_creation_date']).agg({'review_score' : 'mean'}).reset_index()
    df = df.rename(columns = {'review_creation_date' : 'date'})
    df = df[df['date'] >= '2017-04-01 00:00:00']
    df = df[df['date'] < '2018-05-01 00:00:00']
    df['date'] = df['date'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d %H:%M:%S"))
    df['date'] = df['date'].apply(lambda x: datetime.datetime.strftime(x, "%Y-%m-%d"))
    df = df.rename(columns = {"review_score": "csat"})
    return df

In [27]:
# Проверки
res = task_5(reviews)
assert res.date.min() == '2017-04-01'
assert res.date.max() == '2018-04-30'
assert res.csat.sum() == 1551.8881071384853
assert res[res.date == '2017-07-11'].csat.values[0] == 4.291390728476821
assert res[res.date == '2018-02-09'].csat.values[0] == 3.992156862745098
assert res[res.csat == 3.6814814814814816].date.values[0] == '2018-02-25'

## Задание 6 (10 баллов)

Посмотрите, как быстро отвечают пользователи (сделайте аггреграцию по числу дней ответа) и какая средняя оценка

Все манипуляции со временем нужно делать через datetime и dateutil

Табличка, которая у вас должна получиться:

```
days | csat | orders

value | value | value
```

Результаты должны быть отсориртированы по дня по возрастанию

In [ ]:
%%write_and_run task_6.py

# важно! все зависимости, которые используете (если добавляее новые) в этом классе надо явно продублировать в эту ячейку
import numpy as np
import pandas as pd
from dateutil.parser import parse
import datetime

def task_6(reviews):
    df = reviews.copy()
    df['review_answer_timestamp'] = df['review_answer_timestamp'].apply(lambda x : datetime.datetime.strptime(x, "%Y-%m-%d %H:%M:%S"))
    df['review_creation_date'] = df['review_creation_date'].apply(lambda x : datetime.datetime.strptime(x, "%Y-%m-%d %H:%M:%S"))
    df['days'] = df['review_answer_timestamp'] - df['review_creation_date']
    df['days'] = df['days'].apply(lambda x : x.days)
    df = df.groupby(['days']).agg({'order_id' : 'count', 'review_score' : 'mean'}).reset_index()
    df = df.rename(columns = {'review_score' : 'csat', 'order_id' : 'orders'})
    return df

In [ ]:
# Проверки
res = task_6(reviews)
assert res.orders.sum() == 99224
assert np.all(res.days.values == np.sort(res.days.values))
assert len(res) == 214
assert res.days.min() == 0
assert res.days.max() == 518
assert res[res.days == 233].csat.values[0] == 3.0
assert res[res.days == 87].orders.values[0] == 4

## Задание 7 (10 баллов)

Выделите все заказы, где не проставлено поле order_delivered_customer_date. Замените его на дату '2999-12-31'

In [ ]:
%%write_and_run task_7.py

# важно! все зависимости, которые используете (если добавляее новые) в этом классе надо явно продублировать в эту ячейку
import numpy as np
import pandas as pd

def task_7(orders):
    df = orders.copy()
    df['order_delivered_customer_date'] = df['order_delivered_customer_date'].fillna('2999-12-31')
    return df

In [ ]:
#Проверки

res = task_7(orders)
assert len(res[res.order_delivered_customer_date.isna()]) == 0
assert len(res) == 99441
assert len(res[res.order_delivered_customer_date == '2999-12-31']) == 2965

## Задание 8 (30 баллов)

Определите топ-10 продавцов, у которых больше 100 заказов, отсылающие чаще всего свою посылку в другие регионы (считаются только доставленные заказы)

Чаще всего отсылают = самый большой процент отправленных заказов в другой штат

Табличка, которая у вас должна получиться:

```
seller_id | share

value | value
```

In [ ]:
%%write_and_run task_8.py

# важно! все зависимости, которые используете (если добавляее новые) в этом классе надо явно продублировать в эту ячейку
import numpy as np
import pandas as pd

def task_8(orders, items, sellers, customers):
    orders_full = items.join(sellers.set_index('seller_id')[['seller_state']], on='seller_id')
    orders_full = orders_full.join(orders.set_index('order_id')[['customer_id', 'order_status']], on='order_id')
    orders_full = orders_full[orders_full['order_status'] == 'delivered']
    orders_full = orders_full.join(customers.set_index('customer_id')[['customer_state']], on='customer_id')
    orders_full = orders_full[['order_id', 'seller_id', 'customer_id', 'customer_state', 'seller_state']]
    count = orders_full.groupby(['seller_id']).agg({'order_id' : 'nunique'}).reset_index()
    count = count.rename(columns = {'order_id' : 'orders'})
    orders_full = orders_full.join(count.set_index('seller_id')[['orders']], on='seller_id')
    orders_full['other_state'] = (orders_full['seller_state'] != orders_full['customer_state'])
    orders_full = orders_full[orders_full['orders'] > 100]
    count_other = orders_full[orders_full['other_state'] == 1]
    count_other = count_other.groupby(['seller_id']).agg({'order_id' : 'nunique'}).reset_index()
    count_other = count_other.rename(columns = {'order_id' : 'orders_other'})
    orders_full = orders_full.join(count_other.set_index('seller_id')[['orders_other']], on='seller_id').drop_duplicates('seller_id')
    orders_full['share'] = orders_full['orders_other'] / orders_full['orders']
    orders_full = orders_full.sort_values('share', ascending = False)
    orders_full = orders_full[['seller_id', 'share']]
    return orders_full.head(10)

In [ ]:
# Проверки

res = task_8(orders, items, sellers, customers)
assert np.all(res.share.values == np.sort(res.share.values)[::-1])
assert res.share.values[0] == 0.9743589743589743
assert res.share.values[-1] == 0.9356435643564357
assert res.seller_id.values[5] == '1b4c3a6f53068f0b6944d2d005c9fc89'
assert res.seller_id.values[2] == '06a2c3af7b3aee5d69171b0e14f0ee87'
assert res.share.sum() == 9.519118744616716